In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohsin416","key":"0dd970503ddf78155fff07ce1251b711"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle

In [4]:
!kaggle competitions download -c digit-recognizer

  0% 0.00/15.3M [00:00<?, ?B/s]
100% 15.3M/15.3M [00:00<00:00, 1.14GB/s]


In [5]:
!unzip -q digit-recognizer.zip

In [6]:
!ls

digit-recognizer.zip  sample_data	     test.csv
kaggle.json	      sample_submission.csv  train.csv


In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import pandas as pd

In [8]:
training_data = pd.read_csv('train.csv')
training_data[:1]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test_data = pd.read_csv('test.csv')
test_data[:1]

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
x = training_data.iloc[:, 1:].values
y = training_data.iloc[:, 0].values
x_test = test_data.values

In [11]:
x = x.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

In [12]:
x.shape

(42000, 28, 28, 1)

In [13]:
x_test.shape

(28000, 28, 28, 1)

In [14]:
y.shape

(42000,)

In [15]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=10)

In [16]:
y.shape

(42000, 10)

In [17]:
y[:3]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.25, random_state=42)

In [19]:
x_train.shape

(31500, 28, 28, 1)

In [20]:
x_val.shape

(10500, 28, 28, 1)

### CNN

In [21]:
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Dropout, Flatten

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=x_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.30))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.30))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.30))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(.30))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.30))

model.add(Dense(10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,090 (461.29 KB)

 Trainable params: 118,090 (461.29 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [24]:
model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_val, y_val))

Epoch 1/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.5721 - loss: 1.1952 - val_accuracy: 0.9554 - val_loss: 0.1429
Epoch 2/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9290 - loss: 0.2514 - val_accuracy: 0.9735 - val_loss: 0.0861
Epoch 3/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9550 - loss: 0.1644 - val_accuracy: 0.9752 - val_loss: 0.0753
Epoch 4/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9567 - loss: 0.1503 - val_accuracy: 0.9801 - val_loss: 0.0668
Epoch 5/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9648 - loss: 0.1256 - val_accuracy: 0.9775 - val_loss: 0.0769
Epoch 6/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9663 - loss: 0.1195 - val_accuracy: 0.9824 - val_loss: 0.0594
Epoch 7/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9672 - loss: 0.1175 - val_accuracy: 0.9841 - val_loss: 0.0540
Epoch 8/25
985/985 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9708 - loss: 0.1046 - val_accuracy: 0

In [25]:
from sklearn.metrics import classification_report

y_pred_prob = model.predict(x_val)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_val, axis=1)

print(classification_report(y_true, y_pred, digits=4))

329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0     0.9893    0.9922    0.9907      1025
           1     0.9870    0.9974    0.9922      1146
           2     0.9905    0.9711    0.9807      1072
           3     0.9956    0.9852    0.9904      1151
           4     0.9902    0.9873    0.9888      1024
           5     0.9878    0.9944    0.9911       898
           6     0.9921    0.9931    0.9926      1010
           7     0.9774    0.9921    0.9847      1135
           8     0.9871    0.9910    0.9891      1005
           9     0.9922    0.9855    0.9888      1034

    accuracy                         0.9889     10500
   macro avg     0.9889    0.9889    0.9889     10500
weighted avg     0.9889    0.9889    0.9889     10500



In [26]:
model.save(r'digit_recognizer.h5')

In [34]:
x_test.shape

(28000, 28, 28, 1)

In [38]:
y_pred_submit = model.predict(x_test)
y_pred2 = np.argmax(y_pred_submit, axis=1)


df = pd.DataFrame({
    'ImageId': np.arange(1, len(y_pred2) + 1),
    'Label': y_pred2
})

df.to_csv('submission.csv', index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
